In [ ]:
-- Enable service broker to enable communication
ALTER DATABASE [YourDatabase] SET ENABLE_BROKER; 

--Enble message type
CREATE MESSAGE TYPE [YourMessageType]
VALIDATION = NONE;


There are four possible options for the `VALIDATION` clause:

1. **`NONE`**:
    
    - No validation is performed on the message content. This means that any data can be sent as a message, regardless of its format or content.
    - Example:
        ```
        CREATE MESSAGE TYPE [YourMessageType]
        VALIDATION = NONE;
        
        ```
        
2. **`EMPTY`**:
    
    - The message content must be empty. Any message with content will be rejected if this validation type is specified.
    - This is useful for simple control messages where the presence of the message itself (rather than its content) is all that matters.
    - Example:
        ```
        CREATE MESSAGE TYPE [YourMessageType]
        VALIDATION = EMPTY;
        
        ```
        
3. **`WELL_FORMED_XML`**:
    
    - The message content must be well-formed XML. This means the content must follow the rules of XML syntax, but it is not validated against a specific schema.
    - This option is useful if you're exchanging XML data and want to ensure that the XML is correctly structured.
    - Example:
        ```
        CREATE MESSAGE TYPE [YourMessageType]
        VALIDATION = WELL_FORMED_XML;
        
        ```
        
4. **`VALID_XML WITH SCHEMA COLLECTION`**:
    
- The message content must be valid XML that adheres to a specified XML schema collection. This option provides the most rigorous validation, ensuring that the message content matches a predefined schema.
- You must specify the name of an existing XML schema collection in the database.
- Example:
    ```
    sqlCopy codeCREATE MESSAGE TYPE [YourMessageType]
    VALIDATION = VALID_XML WITH SCHEMA COLLECTION YourSchemaCollection;
    
    ```
    
    ### Choosing the Right Validation
    
    - **`NONE`**: Use this when the message content can be any type of data or when you want to perform validation in the application layer rather than within Service Broker.
    - **`EMPTY`**: Use this for simple signaling messages where no content is required.
    - **`WELL_FORMED_XML`**: Use this if you want to ensure the message content is valid XML but don't need it to conform to a specific schema.
    - **`VALID_XML WITH SCHEMA COLLECTION`**: Use this for strict validation against a predefined XML schema, ensuring that the message content is both well-formed and adheres to specific rules.

  

In [ ]:
--Create a contract
CREATE CONTRACT [YourContract]
(YourMessageType SENT BY INITIATOR);

In SQL Server Service Broker, a **contract** defines the types of messages that can be exchanged between two services as part of a conversation. It acts as a formal agreement between the initiating service and the target service, specifying which message types are allowed and who can send them.

### Key Components of a Contract

A contract in Service Broker includes the following components:

1. **Message Type**:
    
    - A message type defines the format and validation rules for a particular kind of message. Contracts reference these message types.
2. **Sender**:
    

- Each message type within a contract is associated with a specific sender, which can either be the **initiator** (the service that starts the conversation) or the **target** (the service that receives the conversation).

### Example of a Contract

Here’s a basic example of how to create a contract in SQL Server Service Broker:

```
CREATE CONTRACT [YourContract]
(
    [YourMessageType1] SENT BY INITIATOR,
    [YourMessageType2] SENT BY TARGET
);

```

In this example:

- **YourContract**: The name of the contract.
- **YourMessageType1**: A message type that the **initiator** service is allowed to send.
- **YourMessageType2**: A message type that the **target** service is allowed to send.

### How Contracts Work

1. **Initiating a Conversation**:
    
    - When a service initiates a conversation, it must specify a contract that defines the message types it will use.
    - The initiator service can only send messages that are defined in the contract and are permitted to be sent by the initiator.
2. **Exchanging Messages**:
    
    - As the conversation progresses, both the initiator and target services can send messages according to the rules set in the contract.
    - If a service tries to send a message type not allowed by the contract, SQL Server will raise an error.
3. **Ending the Conversation**:
    
    - Either service can end the conversation when no more messages need to be exchanged. This usually involves sending an `END CONVERSATION` message.

### Example Scenario

Consider a scenario where you have two services:

- **OrderProcessingService**: Initiates the conversation to process orders.
- **InventoryService**: Handles inventory updates based on the orders.

You might define the following:

1. **Message Types**:
    
    - `OrderRequest`: Sent by the initiator (OrderProcessingService) to request an order to be processed.
    - `OrderResponse`: Sent by the target (InventoryService) to acknowledge the order processing or to provide status updates.
2. **Contract**:
    
    ```
    CREATE CONTRACT [OrderProcessingContract]
    (
        [OrderRequest] SENT BY INITIATOR,
        [OrderResponse] SENT BY TARGET
    );
    
    ```
    

In this setup:

- **OrderProcessingService** is allowed to send `OrderRequest` messages.
- **InventoryService** is allowed to send `OrderResponse` messages.

### Why Contracts Are Important

Contracts enforce rules on the types of messages that can be exchanged and who can send them. This ensures that communication between services is structured and follows a predictable pattern, reducing the likelihood of errors and improving system reliability.

- **Security**: By defining which messages are allowed and who can send them, contracts help prevent unauthorized or unexpected messages from being processed.
- **Clarity**: Contracts make it clear what kind of communication is expected between services, making the system easier to understand and maintain.
- **Validation**: Contracts ensure that the correct message types are used, which can prevent miscommunication or data inconsistencies.

In [ ]:
--Create a queue
CREATE QUEUE [YourQueue];

In [ ]:
--Create your service
CREATE SERVICE [YourService]
ON QUEUE [YourQueue]
(YourContract);


In SQL Server Service Broker, **queues** and **services** are essential components that work together to facilitate reliable, asynchronous messaging between different parts of a system. They play a crucial role in managing and processing messages.

### 1\. **Queue**

A **queue** in Service Broker is a storage structure that holds messages that are sent to a service. Queues ensure that messages are stored securely and are processed in the order they are received, providing a reliable mechanism for message delivery.

#### Key Characteristics of a Queue:

- **Persistent Storage**: Messages are stored in the queue until they are processed, ensuring they are not lost even if the system crashes.
- **FIFO (First-In, First-Out)**: By default, messages are processed in the order they arrive, although you can customize this behavior.
- **Automatic Handling**: SQL Server automatically manages the queue, including storing, retrieving, and maintaining the order of messages.
- **Concurrency Control**: Queues handle concurrent message processing by allowing multiple readers to process messages from the same queue without conflicts.

#### Example of Creating a Queue:

```
CREATE QUEUE [YourQueue];

```

In this example, `YourQueue` is the name of the queue that will hold messages.

### 2\. **Service**

A **service** in Service Broker represents a specific endpoint or entity that sends or receives messages. Services are associated with a queue, and they determine which contract (or contracts) define the types of messages the service can send or receive.

#### Key Characteristics of a Service:

- **Endpoint for Communication**: A service acts as an addressable endpoint that other services can send messages to or receive messages from.
- **Associated with a Queue**: Each service is linked to a specific queue where its messages are stored.
- **Contract Association**: A service operates under one or more contracts, which define the types of messages it can exchange and the rules for communication.

#### Example of Creating a Service:

```
CREATE SERVICE [YourService]
ON QUEUE [YourQueue]
([YourContract]);

```

In this example:

- `YourService` is the name of the service.
- `YourQueue` is the queue where the service’s messages will be stored.
- `YourContract` is the contract that defines the rules for the message exchange.

### How They Work Together:

1. **Sending a Message**:
    
    - When one service (the **initiator**) wants to send a message to another service (the **target**), it does so by starting a conversation using a specific contract.
    - The message is placed in the target service’s queue.
2. **Receiving a Message**:
    
    - The target service retrieves the message from its associated queue. The message stays in the queue until the service processes it.
    - Once processed, the message is typically removed from the queue.
3. **Asynchronous Communication**:
    
- Because the message is stored in a queue, the target service doesn’t need to be immediately available to process the message. This allows for asynchronous communication, where the sender and receiver operate independently.

### Example Scenario

Imagine you have an order processing system where an **OrderService** handles incoming orders, and an **InventoryService** updates inventory based on those orders.

- **OrderService**: Sends an order request message to **InventoryService**.
- **InventoryService**: Receives the message in its queue, processes the order, and sends a confirmation message back.

Here’s how you might set it up:

1. **Create the Queue for InventoryService**:
    
    ```
    CREATE QUEUE [InventoryQueue];
    
    ```
    
2. **Create the Service for InventoryService**:
    
    ```
    CREATE SERVICE [InventoryService]
    ON QUEUE [InventoryQueue]
    ([OrderProcessingContract]);
    
    ```
    
3. **OrderService sends a message**:
    
    - The message is placed in `InventoryQueue`.
    - `InventoryService` retrieves and processes the message.

In [ ]:
--Send a Message to the Queue
--You can send a message to the Service Broker queue from your application or a stored procedure:

DECLARE @conversationHandle UNIQUEIDENTIFIER;
DECLARE @messageBody NVARCHAR(4000) = N'{"Message":"Hello from SQL Server"}';

BEGIN DIALOG CONVERSATION @conversationHandle
    FROM SERVICE [YourService]
    TO SERVICE 'YourService'
    ON CONTRACT [YourContract]
    WITH ENCRYPTION = OFF;

SEND ON CONVERSATION @conversationHandle
    MESSAGE TYPE [YourMessageType]
    (@messageBody);

END CONVERSATION @conversationHandle;